# jupyter notebook in support of [Dangerous Pickles](https://intoli.com/blog/dangerous-pickles/) article by Evan Sangaline.

## simple pickle example:

In [ ]:
import pickle

# start with any instance of a Python type
original = { 'a': 0, 'b': [1, 2, 3] }

# turn it into a string
pickled = pickle.dumps(original)

# turn it back into an identical object
identical = pickle.loads(pickled)

# is the original object identical to the unpickled object
original == identical

## weak pickle bomb

In [ ]:
class Bomb:
    def __init__(self, name):
        self.name = name

    def __getstate__(self):
        return self.name

    def __setstate__(self, state):
        self.name = state
        print(f'Bang! From, {self.name}.')

bomb = Bomb('Evan')

In [ ]:
import pickle

pickled_bomb = pickle.dumps(bomb, protocol=0)
unpickled_bomb = pickle.loads(pickled_bomb)

## inside the [optimized] pickle bomb

In [ ]:
import pickletools

pickled_bomb = pickletools.optimize(pickled_bomb)
pickletools.dis(pickled_bomb)

## Pickle Machine (PM) emulation

In [ ]:
# the PM's longterm memory/storage
memo = {}
# the PM's stack, which most opcodes interact with
stack = []

### translate disassembled pickle to python (instruction by instruction)

In [ ]:
# Push a global object (module.attr) on the stack.
#  0: c    GLOBAL     'copy_reg _reconstructor'
from copyreg import _reconstructor
stack.append(_reconstructor)

# Push markobject onto the stack.
# 25: (    MARK
stack.append('MARK')

# Push a global object (module.attr) on the stack.
# 26: c        GLOBAL     '__main__ Bomb'
stack.append(Bomb)

# Push a global object (module.attr) on the stack.
# 41: c        GLOBAL     '__builtin__ object'
stack.append(object)

# Push None on the stack.
# 61: N        NONE
stack.append(None)

# Build a tuple out of the topmost stack slice, after markobject.
# 62: t        TUPLE      (MARK at 28)
last_mark_index = len(stack) - 1 - stack[::-1].index('MARK')
mark_tuple = tuple(stack[last_mark_index + 1:])
stack = stack[:last_mark_index] + [mark_tuple]

# Push an object built from a callable and an argument tuple.
# 63: R    REDUCE
args = stack.pop()
callable = stack.pop()
stack.append(callable(*args))

# Push a Python Unicode string object.
# 64: V    UNICODE    'Evan'
stack.append(u'Evan')

# Finish building an object, via __setstate__ or dict update.
# 70: b    BUILD
arg = stack.pop()
stack[-1].__setstate__(arg)

# Stop the unpickling machine.
# 71: .    STOP
unpickled_bomb = stack[-1]

### simplified rework

In [ ]:
# Instruction 1, where `_reconstructor` was imported
from copyreg import _reconstructor

# Instruction 7, where `_reconstructor` was called
unpickled_bomb = _reconstructor(cls=Bomb, base=object, state=None)
# Instruction 9, where `__setstate__` was called
unpickled_bomb.__setstate__('Evan')

### and simpler still

In [ ]:
unpickled_bomb = object.__new__(Bomb)
unpickled_bomb.__setstate__('Evan')

## actual pickle bomb

### our bomb in assembly:

### our bomb assembly in pickle machine code:

### our pickle machine code in pickle

In [ ]:
# Run me at home!
# I'm safe, I promise!
pickled_bomb = b'c__builtin__\neval\n(Vprint("Bang! From, Evan.")\ntR.'
pickle.loads(pickled_bomb)

### eval() will take anything...